In [1]:
DIR='/scratch/tm70/as2285/access-om2/archive/'

In [2]:
RUNS = {
    "new_grid_1deg": ["1deg_jra55_ryf-new_grid-3028a7b3"],  # (our name: run name(s))
    "old_grid_1deg": ["1deg_jra55_ryf"],
    "new_grid_025deg" : ["dev-025deg_jra55_ryf-dev-025deg_jra55_ryf-6d175d1a"]
}

In [3]:
for r in RUNS.keys():
    print(r)
    ! cat {DIR}{RUNS[r][0]}/output000/manifests/input.yaml | grep /grid.nc

new_grid_1deg
work/ice/RESTART/grid.nc:
  fullpath: /g/data/tm70/as2285/om3-scripts/grid_generation/1deg/grid.nc
old_grid_1deg
work/ice/RESTART/grid.nc:
  fullpath: /g/data/vk83/experiments/inputs/access-om2/ice/grids/global.1deg/2020.05.30/grid.nc
new_grid_025deg
work/ice/RESTART/grid.nc:
  fullpath: /g/data/tm70/as2285/om3-scripts/grid_generation/025deg/grid.nc


In [4]:
from access_nri_intake.source.builders import AccessOm2Builder
from intake import open_esm_datastore

from dask.distributed import Client

import xarray as xr
from datatree import DataTree, map_over_subtree
import cf_xarray as cfxr


In [5]:
client=Client(threads_per_worker=1)

In [6]:
client

Connection method: Cluster object,Cluster type: distributed.LocalCluster
Dashboard: /proxy/8787/status,
Dashboard: /proxy/8787/status,Workers: 12
Total threads: 12,Total memory: 46.00 GiB
Status: running,Using processes: True
Comm: tcp://127.0.0.1:39275,Workers: 12
Dashboard: /proxy/8787/status,Total threads: 12
Started: Just now,Total memory: 46.00 GiB
Comm: tcp://127.0.0.1:46771,Total threads: 1
Dashboard: /proxy/40825/status,Memory: 3.83 GiB
Nanny: tcp://127.0.0.1:41879,


### Make an esm datastore for each experiment

In [7]:
for s_name,l_name in RUNS.items():
    builder = AccessOm2Builder(DIR+l_name[0])
    builder.build()
    builder.save(s_name, l_name[0])

/g/data/hh5/public/apps/miniconda3/envs/analysis3-24.01/lib/python3.10/site-packages/access_nri_intake/source/utils.py:37: UserWarning: Time coordinate does not include bounds information. Guessing start and end times.
  warnings.warn(
/g/data/hh5/public/apps/miniconda3/envs/analysis3-24.01/lib/python3.10/site-packages/access_nri_intake/source/utils.py:37: UserWarning: Time coordinate does not include bounds information. Guessing start and end times.
  warnings.warn(
/g/data/hh5/public/apps/miniconda3/envs/analysis3-24.01/lib/python3.10/site-packages/access_nri_intake/source/utils.py:37: UserWarning: Time coordinate does not include bounds information. Guessing start and end times.
  warnings.warn(
/g/data/hh5/public/apps/miniconda3/envs/analysis3-24.01/lib/python3.10/site-packages/access_nri_intake/source/utils.py:37: UserWarning: Time coordinate does not include bounds information. Guessing start and end times.
  warnings.warn(
/g/data/hh5/public/apps/miniconda3/envs/analysis3-24.01/

Successfully wrote ESM catalog json file to: file:///home/603/as2285/new_grid_1deg.json


/g/data/hh5/public/apps/miniconda3/envs/analysis3-24.01/lib/python3.10/site-packages/access_nri_intake/source/utils.py:37: UserWarning: Time coordinate does not include bounds information. Guessing start and end times.
  warnings.warn(
/g/data/hh5/public/apps/miniconda3/envs/analysis3-24.01/lib/python3.10/site-packages/access_nri_intake/source/utils.py:37: UserWarning: Time coordinate does not include bounds information. Guessing start and end times.
  warnings.warn(
/g/data/hh5/public/apps/miniconda3/envs/analysis3-24.01/lib/python3.10/site-packages/access_nri_intake/source/utils.py:37: UserWarning: Time coordinate does not include bounds information. Guessing start and end times.
  warnings.warn(
/g/data/hh5/public/apps/miniconda3/envs/analysis3-24.01/lib/python3.10/site-packages/access_nri_intake/source/utils.py:37: UserWarning: Time coordinate does not include bounds information. Guessing start and end times.
  warnings.warn(
/g/data/hh5/public/apps/miniconda3/envs/analysis3-24.01/

Successfully wrote ESM catalog json file to: file:///home/603/as2285/old_grid_1deg.json
Successfully wrote ESM catalog json file to: file:///home/603/as2285/new_grid_025deg.json


/g/data/hh5/public/apps/miniconda3/envs/analysis3-24.01/lib/python3.10/site-packages/access_nri_intake/source/utils.py:37: UserWarning: Time coordinate does not include bounds information. Guessing start and end times.
  warnings.warn(
/g/data/hh5/public/apps/miniconda3/envs/analysis3-24.01/lib/python3.10/site-packages/access_nri_intake/source/utils.py:37: UserWarning: Time coordinate does not include bounds information. Guessing start and end times.
  warnings.warn(


In [8]:
datastores = dict()

In [9]:
for s_name in RUNS.keys():
    datastores[s_name]= open_esm_datastore(
        s_name+'.json', 
        columns_with_iterables=["variable"] 
    )

### Load some data

In [10]:
VARS = ["aice", "hi", "uvel", "vvel" ]  # ice area fraction or sea ice concentration, ice thickness averaged by grid cell area
VARS_2D = ["area_t", "geolat_t", "geolon_t"]

In [11]:
def open_by_name(name, vars):
    """Return a dataset for the requested name and vars"""
    return (
        datastores[name]
        .search(variable=vars)
        .to_dask(
            xarray_open_kwargs={
                "decode_coords": False,
            },
            xarray_combine_by_coords_kwargs={
                "compat": "override",
                "data_vars": "minimal",
                "coords": "minimal",
            },
        )
    )

In [12]:
def open_by_experiment(exp_name, vars):
    """Concatenate any datasets provided for this experiment into one ds, and add area and geo coordinates"""
    from datetime import timedelta

    # get the data for each run of this config
    cice_ds = xr.concat(
        [open_by_name(iName, vars) for iName in [exp_name]], dim="time"
    )

    # We also want the area/lat/lon fields, but these are not time dependent.
    area_ds = xr.merge(
        [
            xr.open_dataset(
                datastores[exp_name]
                .search(variable=iVar)
                .df.path[0]
                # path of the first file with the area field, the geolon field and the geolat field
            ).drop_vars("time")
            for iVar in VARS_2D
        ]
    )

    # Label the lats and lons
    cice_ds.coords["ni"] = area_ds["xt_ocean"].values
    cice_ds.coords["nj"] = area_ds["yt_ocean"].values

    # Copy attributes for cf compliance
    cice_ds.ni.attrs = area_ds.xt_ocean.attrs
    cice_ds.nj.attrs = area_ds.yt_ocean.attrs

    cice_ds = cice_ds.rename(({"ni": "xt_ocean", "nj": "yt_ocean"}))

    # Add the geolon, geolat, and area as extra co-ordinates fields from area_t

    cice_ds = cice_ds.assign_coords(
        {
            "geolat_t": area_ds.geolat_t,
            "geolon_t": area_ds.geolon_t,
            "area_t": area_ds.area_t,
        }
    )

    # cice timestamps are also misleading:
    cice_ds["time"] = cice_ds.time.to_pandas() - timedelta(minutes=1)

    return cice_ds

In [13]:
%%time

si_name_ds_pairs = [(iRun, open_by_experiment(iRun, VARS)) for iRun in RUNS.keys()]

si_dt = DataTree.from_dict(dict(si_name_ds_pairs))

KeyError: 0

# Calculate Sea Ice Area

Sea ice area is the circumpolar sum of sea ice concentration multiplied by the area of each grid cell. By convention, and because lower concentrations are not accurate when measured through remote sensing, concentrations below 0.15 are not included

In [14]:
def sea_ice_area(sic, area, range=[0.15, 1]):
    return (sic * area).where((sic >= range[0]) * (sic <= range[1])).cf.sum(["X", "Y"])

In [15]:
@map_over_subtree
def sea_ice_area_model_sh(ds):
    sic = ds.aice.cf.sel(Y=slice(-90, 0))
    area_km2 = ds.area_t / 1e6

    return sea_ice_area(sic, area_km2).to_dataset(name="si_area").load()

In [16]:
model_area_dt = sea_ice_area_model_sh(si_dt)

NameError: name 'si_dt' is not defined

In [ ]:
for s_name in RUNS.keys():
    model_area_dt[s_name].si_area.plot()

# Some differences in 1 deg conc

In [ ]:
anoms = si_dt['new_grid_1deg']-si_dt['old_grid_1deg']

In [ ]:
anoms.hi.max().values

In [ ]:
anoms.hi.min().values

In [ ]:
anoms.aice.max().values

In [ ]:
anoms.aice.min().values

In [ ]:
anoms.hi.isel(time=50).plot()

In [ ]:
anoms.aice.isel(time=50).plot()

In [ ]:
anoms.aice.isel(time=360).plot()